In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [2]:
from datasets import load_dataset
dataset_xlcost = load_dataset("codeparrot/xlcost-text-to-code", "Python-program-level")['test']
dataset_code_x = load_dataset("google/code_x_glue_ct_code_to_text", 'python')['test']
dataset_code_search_net = load_dataset("code-search-net/code_search_net", 'python')['test']

README.md:   0%|          | 0.00/3.32k [00:00<?, ?B/s]

xlcost-text-to-code.py:   0%|          | 0.00/7.61k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/3.92M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/357k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/200k [00:00<?, ?B/s]

...

The repository for code-search-net/code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code-search-net/code_search_net.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


python.zip:   0%|          | 0.00/941M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [3]:
def split_text(example):
    example['text'] = example['text'].split('|')[0].strip()
    return example

dataset_xlcost = dataset_xlcost.map(split_text)

dataset_code_x = dataset_code_x.rename_column('docstring', 'text')

dataset_code_search_net = dataset_code_search_net.rename_column('func_documentation_string', 'text')
dataset_code_search_net = dataset_code_search_net.rename_column('func_code_string', 'code')

Map:   0%|          | 0/887 [00:00<?, ? examples/s]

In [4]:
!pip install openai

In [5]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="",
)

In [34]:
system_message = {
      "role": "system",
      "content": "For each code snippet provided as input, output a very brief summary and nothing else. The summary must not include newline characters or full stop symbols. The number of summary outputs must exactly match the number of input code snippets, with each summary separated by the '$' symbol if there are multiple snippets. "
    }

In [25]:
def get_summarization(model, dataset, left, right):
    codes = ""
    refs = []
    for sample in dataset.select(range(left,right)):
        codes += sample['code'] + "\n \n"
        refs.append(sample['text'])

    completion = client.chat.completions.create(
        model = model,
        messages=[
          system_message,
          {
            "role": "user",
            "content": codes
          }
        ]
    )
    summary = completion.choices[0].message.content.split('$')

    return summary, refs

In [18]:
!pip install evaluate rouge_score git+https://github.com/google-research/bleurt.git bert_score

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-8y55sruv
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-8y55sruv
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done


In [9]:
import evaluate
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bleurt = evaluate.load("bleurt", module_type="metric", checkpoint="bleurt-base-128")
bertscore = evaluate.load("bertscore")

In [22]:
def calculate_metrics(preds, refs):
    rouge_result = rouge.compute(predictions=preds, references=refs, rouge_types=["rougeL"])["rougeL"]
    bleu_result = bleu.compute(predictions=preds, references=[[ref] for ref in refs])

    bleurt_scores = bleurt.compute(predictions=preds, references=refs)["scores"]
    avg_bleurt = sum(bleurt_scores) / len(bleurt_scores)

    bert_scores = bertscore.compute(
        predictions=preds,
        references=refs,
        lang="en",
        model_type="microsoft/deberta-large-mnli"
    )
    avg_bert_f1 = sum(bert_scores["f1"]) / len(bert_scores["f1"])


    return rouge_result, bleurt_scores, bert_scores, avg_bert_f1

In [39]:
preds, refs = get_summarization("google/gemini-2.0-flash-thinking-exp:free", dataset_xlcost, 0, 10)
rouge_result, bleurt_scores, bert_scores, avg_bert_f1 = calculate_metrics(preds,refs)

print("\rouge_result:", rouge_result)
print("bleurt_scores:", bleurt_scores)
print(f"bert_scores:", bert_scores)
print(f"avg_bert_f1 F1", avg_bert_f1)

find minimum sum by removing elements based on frequency$calculate maximum adjacent difference after removing each element$recursive size calculation and count ones in range$check prime and find minimum divisor based output$calculate Landau function using LCM of array partitions$check primality and print Yes or No$check odd count criteria in array$calculate complex value based on element frequencies and ceilings$insert elements based on XOR and Sum conditions$compare two numbers and print Yes or No
ouge_result: 0.19959805326813002
bleurt_scores: [-0.2413780391216278, 0.11444546282291412, -0.8450124859809875, -0.6083342432975769, -0.8469671010971069, -1.226912498474121, -0.5912383198738098, -0.9825083017349243, -0.4574681520462036, -1.2165277004241943]
bert_scores: {'precision': [0.7296587228775024, 0.6666524410247803, 0.5394080877304077, 0.43860071897506714, 0.5973687171936035, 0.3829222321510315, 0.5596373677253723, 0.47162970900535583, 0.5418701171875, 0.5265482664108276], 'recall': 